In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

import warnings
warnings.filterwarnings(action='ignore')

# CBF (content based filtering)

## 영화 데이터 불러오기

In [2]:
movie=pd.read_csv("movie_0725.csv")
movie.head()


,adult,genres,id,original_language,title
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,Toy Story
1,False,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,Jumanji
2,False,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,Grumpier Old Men
3,False,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,Waiting to Exhale
4,False,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Father of the Bride Part II


In [3]:
type(movie)

pandas.core.frame.DataFrame

In [4]:
movie.shape

(8875, 5)

## genres 컬럼 장르 정보만 남기기

In [5]:
#movie의 장르 부분이 dictionery를 원소로 갖는 리스트인데 마지막에 ""로 문자열로 저장되어 있음을 확인할 수 있다
movie['genres'][0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [6]:
#데이터 타입을 보면 문자열로 되어있음(의도치 않게 전환된 상태)
type(movie['genres'][0])

str

In [7]:
#리스트나 딕셔너리로 저장한 데이터가 문자열로 저장된 것을 다시 리스트 형태로 다시 바꿔주는 작업
from ast import literal_eval

movie["genres"]=movie['genres'].apply(literal_eval)

In [8]:
type(movie['genres'][0])

list

In [9]:
movie['genres'][0] #딕셔너리들을 원소로 갖는 리스트의 형태로 저장되어 있음을 확인할 수 있다

[{'id': 16, 'name': 'Animation'},
 {'id': 35, 'name': 'Comedy'},
 {'id': 10751, 'name': 'Family'}]

In [10]:
#장르값만 남기는 함수(dictionary의 'key인 name'이 가리키는 'value인 장르'만 남기는 함수)
def get_genres(x):
    try:
        genre=[i['name'] for i in x] #genres의 원소들 중 하나를 잡아 해당 원소(딕셔너리 형태)에서 name을 key로 갖는 것의 value값만을 genre에 할당하고
        return genre #반환한다
 
    except:
        genre=[] #해당하는게 없으면 빈 리스트를 할당하고
        return genre #그대로 반환한다

In [11]:
movie["genres"]=movie['genres'].apply(get_genres)
movie

,adult,genres,id,original_language,title
0,False,"[Animation, Comedy, Family]",862,en,Toy Story
1,False,"[Adventure, Fantasy, Family]",8844,en,Jumanji
2,False,"[Romance, Comedy]",15602,en,Grumpier Old Men
3,False,"[Comedy, Drama, Romance]",31357,en,Waiting to Exhale
4,False,[Comedy],11862,en,Father of the Bride Part II
...,...,...,...,...,...
8870,False,[Drama],159550,en,The Last Brickmaker in America
8871,False,"[Adventure, Drama, History, Romance]",402672,hi,Mohenjo Daro
8872,False,[Comedy],97995,en,Seven Years Bad Luck
8873,False,"[Documentary, Music]",391698,en,The Beatles: Eight Days a Week - The Touring Y...


## 장르 공백 제거하기

In [12]:
movie['genres'][21:30] #리스트 안의 원소들은 str이다
#Science Fiction의 경우 중간에 띄어쓰기가 되어 있는 것을 확인할 수 있다 --> 이 공백을 없애보자

21                              [Drama, Thriller]
22           [Action, Adventure, Crime, Thriller]
23    [Drama, Fantasy, Science Fiction, Thriller]
24                               [Drama, Romance]
25                                        [Drama]
26                        [Comedy, Drama, Family]
27                               [Drama, Romance]
28          [Fantasy, Science Fiction, Adventure]
29                                 [Drama, Crime]
Name: genres, dtype: object

In [13]:
### 문자 공백 없애기 

movie["genres"]=movie["genres"].apply(lambda x: [str(i).replace(" ","") for i in x])
movie['genres'][21:30]

21                             [Drama, Thriller]
22          [Action, Adventure, Crime, Thriller]
23    [Drama, Fantasy, ScienceFiction, Thriller]
24                              [Drama, Romance]
25                                       [Drama]
26                       [Comedy, Drama, Family]
27                              [Drama, Romance]
28          [Fantasy, ScienceFiction, Adventure]
29                                [Drama, Crime]
Name: genres, dtype: object

## 리스트에서 텍스트만 추출

In [14]:
def get_text(x):
    return ' '.join(x) #join은 리스트의 요소들을 합쳐서 문자열로 반환해주는 함수

movie['genres']=movie['genres'].apply(get_text)
movie.head()

,adult,genres,id,original_language,title
0,False,Animation Comedy Family,862,en,Toy Story
1,False,Adventure Fantasy Family,8844,en,Jumanji
2,False,Romance Comedy,15602,en,Grumpier Old Men
3,False,Comedy Drama Romance,31357,en,Waiting to Exhale
4,False,Comedy,11862,en,Father of the Bride Part II


In [15]:
type(movie)

pandas.core.frame.DataFrame

## 장르 TF-IDF 계산 

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

#TF-IDF: 각 문서(여기서는 영화 장르)에 대해 단어의 빈도와 문서 빈도의 역수를 곱하여 단어의 상대적인 중요도를 계산하는 방법
tfidf=TfidfVectorizer() #텍스트 데이터를 TF-IDF 행렬로 변환하는 기능을 제공하는 TfidfVectorizer 클래스의 인스턴스를 생성

tfidf_matrix= tfidf.fit_transform(movie['genres']).toarray() #데이터 프레임인 movie의 genres 열의 텍스트 데이터를 TF-IDF행렬로 변환하는 코드
#fit_transform(movie['genres']): TfidfVectorizer의 매서드로 인자로 주어진 데이터(movie['genres'])를 학습하여 TF-IDF 행렬로 변환하는 기능을 수행
#toarray(): TfidfVectorizer의 매서드로 희소 행렬(sparse matrix)를 밀집 행렬(dense matrix)로 변환하는 기능을 수행

tfidf_matrix

array([[0.        , 0.        , 0.72387097, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.52947876, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [17]:
type(tfidf_matrix)

numpy.ndarray

In [18]:
row, column = tfidf_matrix.shape
print(row, column) #row가 8875인 것은 movie의 행이 8875인 것 때문이고, column이 20인 것은 genres에 들어가는 종류(Animation, Family 등등)가 총 20개이기 때문이다

8875 20


## 코사인 유사도 계산

In [19]:
from sklearn.metrics.pairwise import cosine_similarity

cos_matrix = cosine_similarity(tfidf_matrix,tfidf_matrix)
cos_matrix

#주대각선의 유사도는 1이다 --> 자기 자신과의 유사도는 완전히 일치하여 1이라는 것을 확인할 수 있다
#cos_matrix는 각각의 영화(8875개)가 다른 영화(자기 포함 8875개)와 코사인 유사도가 어떻게 되는지를 저장한 행렬이다

array([[1.        , 0.34861102, 0.21362182, ..., 0.35100189, 0.        ,
        0.26805428],
       [0.34861102, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.21362182, 0.        , 1.        , ..., 0.6086059 , 0.        ,
        0.46478216],
       ...,
       [0.35100189, 0.        , 0.6086059 , ..., 1.        , 0.        ,
        0.76368329],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.26805428, 0.        , 0.46478216, ..., 0.76368329, 0.        ,
        1.        ]])

In [20]:
cos_matrix.shape #정방 행렬임을 확인할 수 있다

(8875, 8875)

In [21]:
#각각의 영화가 다른 영화와 코사인 유사도가 어떻게 되는지를 저장한 행렬이기 때문에 cos_matrix로 만든 데이터프레임의 index와 column은 모두 영화를 고유하게 가리키는 movie['id']로 설정한다
df_cosine= pd.DataFrame(data=cos_matrix, index=movie['id'], columns=movie['id']) 
df_cosine.head()

#각 영화가 자기 자신과는 (코사인) 유사도가 1인 것을 확인할 수 있다

id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,399106,368620,401387,314420,390989,159550,402672,97995,391698,265189
id,,,,,,,,,,,,,,,,,,,,,
862,1.000000,0.348611,0.213622,0.189955,0.351002,0.000000,0.213622,0.362279,0.000000,0.000000,...,0.936375,0.0,0.0,0.000000,0.136336,0.000000,0.000000,0.351002,0.0,0.268054
8844,0.348611,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.649309,0.337934,0.337934,...,0.372299,0.0,0.0,0.000000,0.000000,0.000000,0.263262,0.000000,0.0,0.000000
15602,0.213622,0.000000,1.000000,0.889212,0.608606,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.236395,0.000000,0.336923,0.608606,0.0,0.464782
31357,0.189955,0.000000,0.889212,1.000000,0.541180,0.153224,0.889212,0.139394,0.000000,0.000000,...,0.000000,0.0,0.0,0.254105,0.210206,0.457495,0.425556,0.541180,0.0,0.708644
11862,0.351002,0.000000,0.608606,0.541180,1.000000,0.000000,0.608606,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.388421,0.000000,0.000000,1.000000,0.0,0.763683


## 평점 데이터 불러오기

In [22]:
rating= pd.read_csv("rating_0725.csv")

In [23]:
rating #각각의 유저가 각각의 영화에 대해 매긴 rating(평점)을 모두 저장한 데이터

,userId,rating,id
0,1,2.5,9909
1,1,3.0,11360
2,1,3.0,819
3,1,2.0,1103
4,1,4.0,11216
...,...,...,...
99842,671,4.5,25461
99843,671,4.0,51927
99844,671,2.5,604
99845,671,2.5,1088


In [24]:
#평점데이터를 8:2로 train test 데이터로 나우기

from sklearn.model_selection import train_test_split

train_rate, test_rate =train_test_split(rating, test_size=0.2, random_state=100)

In [25]:
train_rate.head()

,userId,rating,id
91215,606,4.5,10315
81219,553,4.0,36557
41536,299,4.5,1103
17858,119,4.0,105
76547,530,3.0,17832


# CBF 평점 공식
### user ID 5가 영화 i에 남길 예상 평점


$$ \hat{r}_{5,i}  =  \frac{\sum_{j\in I_5}{sim(i,j)}\cdot r_{5,j}}{\sum_{j\in I_5}{sim(i,j)}} $$

* $\hat{r}_{5,i}$ 사용자 5가 영화 i에 남길 평점
* $r_{5,j}$ 사용자 5가 영화 j에 남긴 평점
* $sim(i,j)$ 영화 i와 j의 유사도
* $I_5$ 사용자 5가 평점을 남긴 영화 전체 집합 

In [26]:
#train 데이터에서 유저아이디가 5인 데이터 추출
user5_train= train_rate[train_rate['userId'] == 5][['id', 'rating']] 

'''위의 코드는 아래의 두 줄의 코드로 분할해서 생각해볼 수 있다
user5_train = train_rate[train_rate['userId'] == 5] #우선 userId column의 값이 5인 데이터들만 가져온 상태(이 또한 새로운 데이터 프레임)
user5_train = user5_train[['id', 'rating']]         #id, rating 열을 리스트로 묶은 다음(['id', 'rating']) user5_train[ ]에 넣어 해당 열의 데이터만 참조한 데이터 프레임을 만든다
                                                    #[ ] 안에 또 id, rating을 묶은 리스트를 넣기 때문에 [ [ ] ] 꼴이 되는 것이다
'''

user5_train #dtype은 pandas의 데이터 프레임

,id,rating
376,639,4.0
392,8916,4.0
373,510,3.0
423,1430,3.5
396,817,4.0
...,...,...
415,8835,4.5
404,1213,3.5
377,621,5.0
422,4147,4.0


In [27]:
df_cosine.head()

id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,399106,368620,401387,314420,390989,159550,402672,97995,391698,265189
id,,,,,,,,,,,,,,,,,,,,,
862,1.000000,0.348611,0.213622,0.189955,0.351002,0.000000,0.213622,0.362279,0.000000,0.000000,...,0.936375,0.0,0.0,0.000000,0.136336,0.000000,0.000000,0.351002,0.0,0.268054
8844,0.348611,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.649309,0.337934,0.337934,...,0.372299,0.0,0.0,0.000000,0.000000,0.000000,0.263262,0.000000,0.0,0.000000
15602,0.213622,0.000000,1.000000,0.889212,0.608606,0.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.236395,0.000000,0.336923,0.608606,0.0,0.464782
31357,0.189955,0.000000,0.889212,1.000000,0.541180,0.153224,0.889212,0.139394,0.000000,0.000000,...,0.000000,0.0,0.0,0.254105,0.210206,0.457495,0.425556,0.541180,0.0,0.708644
11862,0.351002,0.000000,0.608606,0.541180,1.000000,0.000000,0.608606,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.388421,0.000000,0.000000,1.000000,0.0,0.763683


In [28]:
type(user5_train['id'].values), type(user5_train['id'])

(numpy.ndarray, pandas.core.series.Series)

In [29]:
### 공식의 분모 유사도 합 구하기 (user5_train의 79개 영화인 j와 전체 영화 i 에 대해)
sim_sum = df_cosine.loc[user5_train['id'].values, :].sum().values    #유사도 합
'''
1. user5_train['id'].values = 'user5_train' DataFrame의 'id' 열을 numpy 배열로 변환하여 영화들의 id를 가져온다
2. df_cosine.loc[user5_train['id'].values, :] = 데이터 프레임에 loc을 써서 index가 user5_train['id'].values에 있는 값(639, 8916, ...)과 일치하는 모든 행(row)들을 선택해낸다
3. df_cosine.loc[user5_train['id'].values, :].sum() = 데이터 프레임의 선택된 각 행(row)마다 각 column별 값들을 합산하여 반환한다
4. .values = 마지막에 .values를 붙여서 numpy.ndarray(numpy 배열)로 변환한다
'''
sim_sum = sim_sum + 1                                                #분모가 0인경우 발생할 계산오류를 피하기 위해 +1 해줌
sim_sum #numpy 배열로 저장된 상태

array([18.03566517, 12.00646275, 32.29388642, ..., 26.83057516,
        4.33571418, 32.39231754])

In [30]:
#2단계 - 데이터 프레임에 loc을 써서 index가 user5_train['id'].values에 있는 값(639, 8916, ...)과 일치하는 모든 행(row)들을 선택해낸다 
df_cosine.loc[user5_train['id'].values, :] #df_cosine.loc[user5_train['id'].values]와 동일한 결과(, 뒤에 어차피 전체 열(column)을 선택하기 때문에 없어도 된다)

id,862,8844,15602,31357,11862,949,11860,45325,9091,710,...,399106,368620,401387,314420,390989,159550,402672,97995,391698,265189
id,,,,,,,,,,,,,,,,,,,,,
639,0.189955,0.000000,0.889212,1.000000,0.541180,0.153224,0.889212,0.139394,0.000000,0.000000,...,0.000000,0.0,0.000000,0.254105,0.210206,0.457495,0.425556,0.541180,0.000000,0.708644
8916,0.881429,0.557358,0.188292,0.167432,0.309383,0.000000,0.188292,0.579211,0.301451,0.301451,...,0.825348,0.0,0.000000,0.000000,0.120171,0.000000,0.234841,0.309383,0.000000,0.236271
510,0.000000,0.000000,0.000000,0.457495,0.000000,0.334921,0.000000,0.304690,0.000000,0.000000,...,0.000000,0.0,0.000000,0.555428,0.000000,1.000000,0.275326,0.000000,0.000000,0.645591
1430,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.704743,0.000000
817,0.120664,0.277878,0.209221,0.186042,0.343771,0.308552,0.209221,0.288773,0.334957,0.334957,...,0.000000,0.0,0.581945,0.000000,0.516174,0.000000,0.260943,0.343771,0.000000,0.262532
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8835,0.351002,0.000000,0.608606,0.541180,1.000000,0.000000,0.608606,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.388421,0.000000,0.000000,1.000000,0.000000,0.763683
1213,0.000000,0.000000,0.000000,0.180677,0.000000,0.848057,0.000000,0.120330,0.312856,0.312856,...,0.000000,0.0,0.000000,0.711032,0.000000,0.394927,0.108734,0.000000,0.000000,0.254961
621,0.000000,0.000000,0.793473,0.705566,0.000000,0.000000,0.793473,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.424618,0.000000,0.000000,0.000000


In [31]:
#3단계 - 데이터 프레임의 선택된 각 행(row)마다 각 column별 값들을 합산하여 반환한다
df_cosine.loc[user5_train['id'].values, :].sum() #.values로 numpy 배열로 변환하지 않으면 pandas의 Series 객체로 남아있는다

id
862       17.035665
8844      11.006463
15602     31.293886
31357     36.093916
11862     25.830575
            ...    
159550    18.070168
402672    16.816537
97995     25.830575
391698     3.335714
265189    31.392318
Length: 8875, dtype: float64

In [32]:
#공식의 분자 계산 - 각각 곱하고 더한 값이 필요하므로 내적 함수사용  
sim_rating = np.matmul(df_cosine.loc[user5_train['id'].values, :].T.values, user5_train['rating'].values)
'''
두 벡터 a1, a2의 내적 = a1.T ∙ a2 이므로 
1. df_cosine.loc[user5_train['id'].values, :].T.values = 유사도(의 transpose)
2. user5_train['rating'].values = 사용자 5가 영화에 남긴 평점
3. np.matmul = 두 배열(행렬)의 행렬 곱셈을 수행하는 함수(내적 계산(A∙B)을 위한 함수)
'''
sim_rating

array([ 67.35395135,  42.83755096, 126.84272225, ..., 103.06395446,
        14.06628734, 124.02959458])

In [33]:
df_cosine.loc[user5_train['id'].values, :].T.values.shape

(8875, 79)

In [34]:
user5_train['rating'].values.shape

(79,)

In [35]:
sim_sum.shape, sim_rating.shape #분자인 sim_sum, 분모인 sim_rating 모두 8875개의 데이터로 이루어진 numpy 배열

((8875,), (8875,))

In [36]:
#최종 평점 예측
#np.divide: NumPy 라이브러리에서 제공하는 요소별(divide element-wise)로 배열의 값을 나누는 함수, 배열의 각 요소들을 서로 나누어 새로운 배열을 생성
pred_rating = pd.DataFrame(np.divide(sim_rating, sim_sum), index=df_cosine.index)
pred_rating.columns=["pred"]
pred_rating

,pred
id,
862,3.734487
8844,3.567874
15602,3.927763
31357,3.906486
11862,3.841288
...,...
159550,3.681214
402672,3.772871
97995,3.841288


## 예측한 평점 성능평가

In [37]:
#이전에는 train_rate, 즉 train set를 기반으로 평점 예측을 한 것이고 이번에 다루는 것은 test set의 데이터이다
user5_test= test_rate[test_rate['userId'] == 5][['id', 'rating']]
user5_test.head()

,id,rating
436,558,4.5
428,310,4.0
374,240,2.5
417,161,3.0
425,672,3.0


In [38]:
user5= pd.merge(user5_test, pred_rating.loc[user5_test['id']], left_on = 'id',right_index=True)
'''
1. merge함수를 통해 앞에 오는 두 개의 데이터 프레임을 합친다
2. pred_rating에서 user5_test의 'id' 열에 해당하는 값과 같은 열들만을 뽑아 두 번째 인자(데이터 프레임)로 넣는다
3. left_on = 'id'는 첫 번째 인자인 user5_test의 'id'열을 기준으로 사용하겠다는 뜻
4. right_index = True는 두 번째 인자인 pred_rating.loc[user5_test['id']]에서 선택된 행들의 인덱스를 기준으로 합치겠다는 뜻 
'''
user5.head()

,id,rating,pred
436,558,4.5,3.491039
428,310,4.0,3.739688
374,240,2.5,3.446859
417,161,3.0,3.045749
425,672,3.0,3.567874


In [39]:
### 평점을 잘 예측했는지 성능 평가 함수

def evaluation(pred, rating):
    rmse=np.round(math.sqrt(np.mean((pred-rating)**2)),3)   #rmse: 오차를 제곱한 값의 평균에 루트를 씌운 값
    mae=np.mean(np.abs(pred-rating)).round(3)               #mae: 오차의 절댓값의 평균
    
    print('RMSE:', rmse)
    print('MAE:', mae)
    
evaluation(user5['rating'],user5['pred'])

#장르라는 특성만을 사용해서 정확도가 떨어지는 경향이 있는 듯 하다 

RMSE: 0.956
MAE: 0.58


## 유사한 영화를 알려주는 함수 만들기

In [40]:
movie.index

RangeIndex(start=0, stop=8875, step=1)

In [41]:
### 어떤 영화 제목을 입력하면 그와 유사도가 높은 10개 영화와 각 예상 평점 나오도록 하는 함수 만들기
indices = pd.Series(movie.index , index=movie['title']) #영화의 제목을 인덱스로 하는 시리즈 객체를 indices에 저장
indices

title
Toy Story                                                0
Jumanji                                                  1
Grumpier Old Men                                         2
Waiting to Exhale                                        3
Father of the Bride Part II                              4
                                                      ... 
The Last Brickmaker in America                        8870
Mohenjo Daro                                          8871
Seven Years Bad Luck                                  8872
The Beatles: Eight Days a Week - The Touring Years    8873
Force Majeure                                         8874
Length: 8875, dtype: int64

In [42]:
indices["Toy Story"] #특정 영화 제목에 해당하는 index 값을 반환하는 것을 확인할 수 있다

0

In [43]:

def similar_movie_1(title, cos_matrix):
    idx = indices[title]                #제목을 입력하면 해당 인덱스 값(몇번째 행인지)을 가져옴
    
    cos_sim = list(enumerate(cos_matrix[idx]))          #몇번째 행인지와 유사도를 같이 가져옴(8875x8875인 cos_matrix에서 idx에 맞는 행의 데이터(1x8875)를
                                                        #cos_matrix의 인덱스(영화의 id)와 그 영화와의 유사도를 튜플 형태로 반환(2x8875)한 것을 리스트로 변환
                                                        #cf) enumerate: 각 요소와 해당 요소의 인덱스를 튜플 형태로 반환하는 함수
#    return cos_sim
    cos_sim.sort(key=lambda x :x[1], reverse=True)   #유사도 값을 기준으로 내림차순 정렬(두 번째 column이 유사도이기 때문에 x[1]을 기준으로 하고 reverse=True로 내림차순 정렬)
    
    cos_sim = cos_sim[0:11]                          #상위 11개 작품 (다음 코드에서 입력한 작품 제외할 것)   
    
    sim_movie_idx=[x[0] for x in cos_sim]            #cos_sim에서 첫번째 항목만 즉, 몇 행인지 값 추출(인덱스와 유사도의 튜플에서 인덱스(곧 영화의 id)만 추출한다는 것)
    
    return cos_sim
#     if idx in sim_movie_idx:                         #해당 작품 제외(주어진 title에 해당하는 작품이 상위 11개의 작품의 인덱스와 동일한 것은 리스트에서 제거(remove 사용))
#         sim_movie_idx.remove(idx)
              
#     sim_movie_idx = sim_movie_idx[0:10]              #상위 10개 작품만
# #     return sim_movie_idx
    
#     title_list=movie['title'].iloc[sim_movie_idx]  #인덱스로 유사한 영화 제목 추출
#     title_list=pd.DataFrame(title_list)            #데이터프레임 만듦
# #     return title_list
    
#     id_list=movie['id'].iloc[sim_movie_idx]        #인덱스로 영화 id 추출
#     id_list=pd.DataFrame(id_list)                  #데이터 프레임으로 만듦
# #     return id_list

#     temp1 = pd.concat([title_list,id_list], axis=1)                              #title_list, id_list 데이터프레임 병합
# #     return temp1
#     temp2=pd.merge(temp1, pred_rating["pred"],left_on = 'id',right_index=True)   #temp1과 pred_rating 병합

#     result=temp2.drop(["id"], axis=1)
#     result.rename(columns={"title":"유사한 영화 TOP10", "pred":"예상평점"},inplace=True)
    
#     print(result)

    
   

In [44]:
indices["The City of Lost Children"]

28

In [45]:
similar_movie_1("The City of Lost Children",cos_matrix)

[(28, 0.9999999999999999),
 (5977, 0.9999999999999999),
 (6713, 0.9999999999999999),
 (6721, 0.9999999999999999),
 (7756, 0.9999999999999999),
 (3354, 0.9596142084424715),
 (5166, 0.9596142084424715),
 (1259, 0.9071666519946471),
 (2089, 0.9071666519946471),
 (2090, 0.9071666519946471),
 (3059, 0.9071666519946471)]

In [46]:
similar_movie_1("Father of the Bride Part II",cos_matrix)

[(4, 1.0),
 (58, 1.0),
 (60, 1.0),
 (63, 1.0),
 (80, 1.0),
 (86, 1.0),
 (94, 1.0),
 (117, 1.0),
 (118, 1.0),
 (120, 1.0),
 (133, 1.0)]

In [47]:
similar_movie_1("Jumanji",cos_matrix)

[(1, 1.0),
 (55, 1.0),
 (729, 1.0),
 (1508, 1.0),
 (1632, 1.0),
 (3763, 1.0),
 (4130, 1.0),
 (4258, 1.0),
 (4460, 1.0),
 (4951, 1.0),
 (5131, 1.0)]

In [48]:
# similar_movie_1("Sabrina",cos_matrix)   #영화제목은 같으나 영화id가 다른 경우 에러 발생

In [49]:
indices["Sabrina"]

title
Sabrina      6
Sabrina    725
dtype: int64

In [50]:
#영화 제목은 같은데 영화 id는 다를경우 위 함수로는 에러가 남. 영화 id가 다르면 다른 영화로 보고 모두 출력하는 함수 만들기

def similar_movie(title, cos_matrix):
    #기존 코드는 모두 try에 넣고
    try:
        
        idx = indices[title]                #제목을 입력하면 해당 인덱스 값(몇번째 행인지)을 가져옴
    
        cos_sim = list(enumerate(cos_matrix[idx]))         #몇번째 행인지와 유사도를 같이 가져옴
        cos_sim.sort(key=lambda x :x[1], reverse=True)    #유사도 값을 기준으로 내림차순 정렬

        cos_sim = cos_sim[0:11]                           #상위 11개 작품 (다음 코드에서 입력한 작품 제외할 것)   
        sim_movie_idx=[x[0] for x in cos_sim]            #cos_sim에서 첫번쨰 항목만 즉, 몇행인지 값 추출
        sim_movie_idx.remove(idx)                        #해당 작품 제외
        sim_movie_idx = sim_movie_idx[0:10]              #상위 10개 작품만


        title_list=movie['title'].iloc[sim_movie_idx]  #인덱스로 유사한 영화 제목 추출
        title_list=pd.DataFrame(title_list)           #데이터프레임 만듦

        id_list=movie['id'].iloc[sim_movie_idx]     #인덱스로 영화 id 추출
        id_list=pd.DataFrame(id_list)  #데이터 프레임으로 만듦

        temp1 = pd.concat([title_list,id_list], axis=1)   #title_list, id_list 데이터프레임 병합
        temp2=pd.merge(temp1, pred_rating["pred"],left_on = 'id',right_index=True)    #temp1과 pred_rating 병합

        result=temp2.drop(["id"], axis=1)
        result.rename(columns={"title":"유사한 영화 TOP10", "pred":"예상평점"},inplace=True)

        print(result)

    #같은 title에 여러 개의 영화(다른 id)가 있는 경우를 처리해주는 코드를 except 부분에 작성한다
    except:
        idx=[]
        if len(indices[title])>1: #제목으로 찾은 영화의 index가 2개 이상인 경우

            for i in range(len(indices[title])):
                idx.append(indices[title][i])                          #같은 제목중 코드 하나만 

                #기존에 idx로 사용하던 부분을 idx[i]로 대체하여 사용하게 된다
                
                cos_sim = list(enumerate(cos_matrix[idx[i]]))         #몇번째 행인지와 유사도를 같이 가져옴
                cos_sim.sort(key=lambda x :x[1], reverse=True)    #유사도 값을 기준으로 내림차순 정렬


                cos_sim = cos_sim[0:11]                         #상위 11개 작품 (다음 코드에서 입력한 작품 제외할 것)   
                sim_movie_idx=[x[0] for x in cos_sim]            #cos_sim에서 첫번쨰 항목만 즉, 몇행인지 값 추출


                if idx[i] in sim_movie_idx:
                    sim_movie_idx.remove(idx[i])                    #해당 작품 제외

                sim_movie_idx = sim_movie_idx[0:10]              #상위 10개 작품만



                title_list = movie['title'].iloc[sim_movie_idx]  #인덱스로 유사한 영화 제목 추출
                title_list=pd.DataFrame(title_list)



                id_list=movie['id'].iloc[sim_movie_idx]     #인덱스로 영화 id 추출
                id_list=pd.DataFrame(id_list)


                temp1 = pd.concat([title_list,id_list], axis=1)

                temp2=pd.merge(temp1, pred_rating["pred"],left_on = 'id',right_index=True)    #temp1과 pred_rating 병합



                result=temp2.drop(["id"], axis=1)
                result.rename(columns={"title":f"영화 id:{idx[i]} 유사한 영화 TOP10", "pred":"예상평점"},inplace=True)
                
                print(result)


In [51]:
similar_movie("Sabrina",cos_matrix)

       영화 id:6 유사한 영화 TOP10      예상평점
2          Grumpier Old Men  3.927763
48         Mighty Aphrodite  3.927763
59            Two If by Sea  3.927763
62             French Twist  3.927763
92                Mr. Wrong  3.927763
105            If Lucy Fell  3.927763
108               Boomerang  3.927763
111  Flirting with Disaster  3.927763
113          Pie in the Sky  3.927763
157                Mallrats  3.927763
       영화 id:725 유사한 영화 TOP10      예상평점
3           Waiting to Exhale  3.906486
10     The American President  3.906486
36                   Clueless  3.906486
52      Home for the Holidays  3.906486
53                The Postman  3.906486
66      Kicking and Screaming  3.906486
84            Beautiful Girls  3.906486
109         Chungking Express  3.906486
121     The Brothers McMullen  3.906486
125  An Awfully Big Adventure  3.906486


In [52]:
similar_movie("Alice in Wonderland",cos_matrix) #이상한 나라의 앨리스는 동일한 이름으로 3개의 영화가 있다는 것을 알 수 있다

                             영화 id:825 유사한 영화 TOP10      예상평점
1629                                   The Rescuers  3.574249
2217         Little Nemo: Adventures In Slumberland  3.574249
3901                           Return to Never Land  3.574249
4167                                  Spirited Away  3.574249
5522                              The Polar Express  3.574249
5769                      Kirikou and the Sorceress  3.574249
6444                      Arthur and the Invisibles  3.574249
7316                       How to Train Your Dragon  3.574249
7439  Legend of the Guardians: The Owls of Ga'Hoole  3.574249
8117                                           Epic  3.574249
                       영화 id:7293 유사한 영화 TOP10      예상평점
1                                      Jumanji  3.567874
55                  The Indian in the Cupboard  3.567874
729                           The Wizard of Oz  3.567874
1508                                 Labyrinth  3.567874
1632                             

# User based CF (collaborative filtering)

In [59]:
#평점데이터와 영화데이터의 id, title 병합하기
df= pd.merge(rating, movie[["id","title"]], on="id") #id를 기준으로 rating과 movie를 합친다
df

,userId,rating,id,title
0,1,2.5,9909,Dangerous Minds
1,7,3.0,9909,Dangerous Minds
2,31,4.0,9909,Dangerous Minds
3,32,4.0,9909,Dangerous Minds
4,36,3.0,9909,Dangerous Minds
...,...,...,...,...
98550,664,3.0,34812,H.G. Wells' War of the Worlds
98551,664,4.5,22825,The Box
98552,665,4.0,110972,Pie in the Sky
98553,665,3.0,26602,Summer Catch


In [63]:
#각 유저가 영화에 남긴 평점들로 이루어진 데이터 프레임 만들기
#pivot_table: 주어진 데이터프레임의 값(rating)을 기반으로 새로운 인덱스(userId)와 컬럼(id = 각 영화)에 따라 셀을 정렬
df3=df.copy().pivot_table("rating",index="userId",columns="id")
df3.fillna(0, inplace=True)  #fillna: 누락된 값은 0으로 채우고 inplace=True로 df3에 변경사항을 저장하라는 것
df3

id,2,5,6,11,12,13,14,15,16,18,...,376570,384798,387893,390989,391698,399106,401387,402672,410921,416437
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,3.0,5.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## user-based cf 유사도 계산

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

cos_matrix_3 = cosine_similarity(df3.values, df3.values)
cos_matrix_3 #유저간의 유사도가 계산되었다

array([[1.        , 0.        , 0.        , ..., 0.0480577 , 0.        ,
        0.02367533],
       [0.        , 1.        , 0.12682458, ..., 0.02478699, 0.19422398,
        0.09883407],
       [0.        , 0.12682458, 1.        , ..., 0.08253112, 0.12760636,
        0.17510851],
       ...,
       [0.0480577 , 0.02478699, 0.08253112, ..., 1.        , 0.05424404,
        0.05384711],
       [0.        , 0.19422398, 0.12760636, ..., 0.05424404, 1.        ,
        0.1614514 ],
       [0.02367533, 0.09883407, 0.17510851, ..., 0.05384711, 0.1614514 ,
        1.        ]])

In [62]:
cos_matrix_3.shape #671명의 유저에 대한 정방 행렬이므로 671 x 671 꼴이다

(671, 671)

In [65]:
df3_cosine= pd.DataFrame(data=cos_matrix_3, index=df3.index, columns=df3.index) #userId를 인덱스와 컬럼으로 해서 cos_matrix를 다시 데이터 프레임화 한다
df3_cosine

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074714,0.016818,0.000000,0.084825,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.018850,0.048426,0.000000,0.000000,0.000000,0.048058,0.000000,0.023675
2,0.000000,1.000000,0.126825,0.120170,0.104498,0.000000,0.197844,0.114630,0.114265,0.043568,...,0.448826,0.020123,0.073289,0.161596,0.444087,0.481427,0.005611,0.024787,0.194224,0.098834
3,0.000000,0.126825,1.000000,0.082879,0.153354,0.061421,0.158332,0.253912,0.136093,0.116052,...,0.149219,0.024364,0.160037,0.115863,0.180214,0.177542,0.000000,0.082531,0.127606,0.175109
4,0.074714,0.120170,0.082879,1.000000,0.131055,0.079896,0.324338,0.192461,0.030512,0.137612,...,0.106947,0.016478,0.138503,0.244845,0.093511,0.090381,0.073517,0.095145,0.067317,0.215614
5,0.016818,0.104498,0.153354,0.131055,1.000000,0.063796,0.096964,0.166451,0.086616,0.032370,...,0.201964,0.000000,0.157836,0.185140,0.141644,0.058810,0.044216,0.039065,0.084830,0.221317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.481427,0.177542,0.090381,0.058810,0.000000,0.256565,0.095494,0.102470,0.057056,...,0.410318,0.034160,0.077470,0.154941,0.344634,1.000000,0.019051,0.034715,0.193229,0.130597
668,0.000000,0.005611,0.000000,0.073517,0.044216,0.025189,0.026237,0.039900,0.137311,0.083474,...,0.000000,0.000000,0.060893,0.052358,0.000000,0.019051,1.000000,0.000000,0.068219,0.094058
669,0.048058,0.024787,0.082531,0.095145,0.039065,0.041728,0.068092,0.047863,0.044747,0.059797,...,0.033136,0.015104,0.058473,0.102318,0.019535,0.034715,0.000000,1.000000,0.054244,0.053847


# user-based CF 평점 공식
$$ \hat{r}_{5,i}  = \bar{r_5}+ \frac{\sum_{v\in U_i}(r_{v,i}- \bar{r_v})\cdot{sim(5,v)} }{\sum_{v\in U_i}{sim(5,v)}} $$

* $\hat{r}_{5,i}$ 사용자 5가 영화 i에 남길 평점
* $\bar{r_5}$ 사용자 5가 영화에 남긴 평점들의 평균
* $sim(5,v)$ 사용자 5와 v의 유사도
* $U_i$ 영화 i에 대해 평점을 남긴 사용자 전체 집합

In [67]:
#df3에 대해 0과 1로 이루어진 데이터프레임 만들기 ()
bin_df=df3.copy() #복사해와서
bin_df[bin_df==0.0]=0.0 #0인 것은 0으로 두고
bin_df[bin_df!=0.0]=1.0 #0이 아닌 것은 1로 변환 --> 개수를 세려고
bin_df.head(3)

id,2,5,6,11,12,13,14,15,16,18,...,376570,384798,387893,390989,391698,399106,401387,402672,410921,416437
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
#유저가 평점을 남긴 영화의 개수 (즉, bin_df에서 1의 개수)
bin_sum= bin_df.sum(axis=1) #각 행마다(axis = 1) 모든 값을 다 더하면 1이 있는 셀의 개수가 되고 이것이 해당 유저가 평점을 남긴 영화의 개수가 된다
bin_sum

userId
1       20.0
2       75.0
3       50.0
4      203.0
5      100.0
       ...  
667     67.0
668     18.0
669     36.0
670     29.0
671    111.0
Length: 671, dtype: float64

In [69]:
#유저별 남긴 평점 평균 구하기
#평균 구하기
rating_average= pd.DataFrame(df3.sum(axis=1).divide(bin_sum)) # 각 유저별 평점들의 합을 평점을 준 영화 개수로 나누어 '평점의 평균'을 구한다
rating_average.columns=['average']
rating_average.head()

,average
userId,
1,2.550000
2,3.480000
3,3.560000
4,4.344828
5,3.910000


### user based CF 평점 공식 분자 구하기
$$\sum_{v\in U_i}(r_{v,i}- \bar{r_v})\cdot{sim(5,v)}$$

In [70]:
df3_cosine

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074714,0.016818,0.000000,0.084825,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.018850,0.048426,0.000000,0.000000,0.000000,0.048058,0.000000,0.023675
2,0.000000,1.000000,0.126825,0.120170,0.104498,0.000000,0.197844,0.114630,0.114265,0.043568,...,0.448826,0.020123,0.073289,0.161596,0.444087,0.481427,0.005611,0.024787,0.194224,0.098834
3,0.000000,0.126825,1.000000,0.082879,0.153354,0.061421,0.158332,0.253912,0.136093,0.116052,...,0.149219,0.024364,0.160037,0.115863,0.180214,0.177542,0.000000,0.082531,0.127606,0.175109
4,0.074714,0.120170,0.082879,1.000000,0.131055,0.079896,0.324338,0.192461,0.030512,0.137612,...,0.106947,0.016478,0.138503,0.244845,0.093511,0.090381,0.073517,0.095145,0.067317,0.215614
5,0.016818,0.104498,0.153354,0.131055,1.000000,0.063796,0.096964,0.166451,0.086616,0.032370,...,0.201964,0.000000,0.157836,0.185140,0.141644,0.058810,0.044216,0.039065,0.084830,0.221317
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.481427,0.177542,0.090381,0.058810,0.000000,0.256565,0.095494,0.102470,0.057056,...,0.410318,0.034160,0.077470,0.154941,0.344634,1.000000,0.019051,0.034715,0.193229,0.130597
668,0.000000,0.005611,0.000000,0.073517,0.044216,0.025189,0.026237,0.039900,0.137311,0.083474,...,0.000000,0.000000,0.060893,0.052358,0.000000,0.019051,1.000000,0.000000,0.068219,0.094058
669,0.048058,0.024787,0.082531,0.095145,0.039065,0.041728,0.068092,0.047863,0.044747,0.059797,...,0.033136,0.015104,0.058473,0.102318,0.019535,0.034715,0.000000,1.000000,0.054244,0.053847


In [71]:
#유저 아이디 5와 다른 유저들의 유사도   sim(5,v)
sim_user5 = df3_cosine.loc[5, :] #코사인 유사도에서 5번째 행(row)의 모든 열(column)의 값을 추출하여 sim_user5에 할당 --> 유저 5와 다른 유저들 사이의 유사도를 나타내는 데이터
sim_user5

userId
1      0.016818
2      0.104498
3      0.153354
4      0.131055
5      1.000000
         ...   
667    0.058810
668    0.044216
669    0.039065
670    0.084830
671    0.221317
Name: 5, Length: 671, dtype: float64

In [72]:
sim_user5[5] = 0 #유저 5자신 값 0으로 바꿈

In [73]:
#최종 sim(5,v) 
sim_user5

userId
1      0.016818
2      0.104498
3      0.153354
4      0.131055
5      0.000000
         ...   
667    0.058810
668    0.044216
669    0.039065
670    0.084830
671    0.221317
Name: 5, Length: 671, dtype: float64

In [74]:
#이제 'r(v, i) - r(v)의 평균', 즉 'i번째 영화에 유저5가 남긴 평점 - 유저5의 평점의 평균'을 구해야 한다
#그러나 전자의 df3와 후자의 rating_average의 shape이 달라서 행렬 연산이 불가능하다 --> bin_df 활용 크기 맞추어 계산한다

print(df3.shape)
print(rating_average.shape)
print(bin_df.shape)

(671, 8834)
(671, 1)
(671, 8834)


In [79]:
bin_df.T

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
12,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [81]:
bin_df.T.multiply(rating_average['average']).T
#bin_df.T의 column이 userId(유저)를 뜻하고 이에 행렬 곱을 하는 rating_average의 row가 userId(유저)를 뜻하니 행렬 곱이 가능하다 --> 결과로 각 영화에 대해 userId의 평균 평점이 기록된다
#한 행을 기준으로 같은 userId(유저)가 평점을 매긴(0이 아닌 셀) 영화(id)에 대해서는 미리 구해둔 rating_average의 값(평점의 평균)이 동일하게 들어간 것을 확인할 수 있다

id,2,5,6,11,12,13,14,15,16,18,...,376570,384798,387893,390989,391698,399106,401387,402672,410921,416437
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.00,3.480000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,3.56,3.560000,3.560000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,4.344828,0.00,4.344828,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.000000,3.91,3.910000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.000000,0.00,3.641791,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,3.833333,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [84]:
#유사한 유저들을 곱해 공식 분자 식 완성
user_simrating = (df3 - bin_df.T.multiply(rating_average['average']).T).T.multiply(sim_user5).T.sum(axis=0)  
#df3 : i번째 영화에 각 유저가 매긴 평점
#bin_df.T.multiply(rating_average['average'].T) : 각 유저가 매긴 평점의 평균
#(df3 - bin_df.T.multiply(rating_average['average']).T) : 671x8834 행렬이므로 sim_user5(671x671)과 내적을 구하기 위해서는 Transpose했다가 내적하고 다시 Transpose해야 한다
#그 후 열(column)을 기준으로(axis = 0)합산한다

In [83]:
(df3 - bin_df.T.multiply(rating_average['average']).T).T.multiply(sim_user5).T

id,2,5,6,11,12,13,14,15,16,18,...,376570,384798,387893,390989,391698,399106,401387,402672,410921,416437
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.000000,-0.050159,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,-0.085878,0.220830,0.067476,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.085864,0.000000,0.085864,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.0,0.0,0.0,0.000000,0.000000,0.079877,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
668,0.0,0.0,0.0,-0.036847,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
user_simrating     #영화별로 값이 나옴

id
2          0.013414
5          0.040713
6         -0.077773
11        11.898586
12         2.022910
            ...    
399106     0.087097
401387    -0.400894
402672     0.057055
410921    -0.121685
416437     0.102452
Length: 8834, dtype: float64

### user based CF 평점 공식 분모 계산
$$\sum_{v\in U_i}{sim(5,v)}$$

In [86]:
#유사도의 합
sim_sum = bin_df.T.multiply(sim_user5).T.sum(axis=0)

In [87]:
sim_sum     #이 값도 영화별로 나옴  최종적으로 구하는 것이 영화별 평점 예측이니까 

id
2          0.023691
5          3.111457
6          0.586714
11        34.680364
12        21.804937
            ...    
399106     0.137344
401387     0.189209
402672     0.064925
410921     0.169107
416437     0.097541
Length: 8834, dtype: float64

### user based CF 평점 공식 최종 계산
$$ \hat{r}_{5,i}  = \bar{r_5}+ \frac{\sum_{v\in U_i}(r_{v,i}- \bar{r_v})\cdot{sim(5,v)} }{\sum_{v\in U_i}{sim(5,v)}} $$

In [89]:
#최종 평점 예측 값
pred_rating = rating_average.loc[5].values + pd.Series(data = np.divide(user_simrating,sim_sum), name='pred') #name으로 시리즈의 이름을 지정하여 열 레이블이 된다
pred_rating.fillna(0, inplace=True)

pred_rating

id
2         4.476225
5         3.923085
6         3.777443
11        4.253093
12        4.002773
            ...   
399106    4.544152
401387    1.791209
402672    4.788788
410921    3.190421
416437    4.960347
Name: pred, Length: 8834, dtype: float64

## user based CF 성능평가

In [90]:
user5_test= test_rate[test_rate['userId'] == 5][['id', 'rating']]
user5_test.head(3)

,id,rating
436,558,4.5
428,310,4.0
374,240,2.5


In [91]:
user_temp =pd.merge(user5_test, pred_rating.loc[user5_test['id']], left_on = 'id',right_index=True)
user_temp

,id,rating,pred
436,558,4.5,4.014069
428,310,4.0,3.583973
374,240,2.5,4.594790
417,161,3.0,4.074043
425,672,3.0,3.830929
375,37247,4.0,4.265914
411,8358,3.5,3.867502
414,824,3.5,4.110406
412,1493,4.5,3.445898
388,9552,3.5,3.946920


In [92]:
# 오차 확인
evaluation(user_temp['pred'],user_temp['rating'])

RMSE: 0.744
MAE: 0.58


# 💻 과제   Item-based CF (collaborative filtering)

In [94]:
df= pd.merge(rating, movie[["id","title"]], on="id") #id를 기준으로 rating과 movie의 선택 열들을 합친 데이터 프레임을 생성
df

,userId,rating,id,title
0,1,2.5,9909,Dangerous Minds
1,7,3.0,9909,Dangerous Minds
2,31,4.0,9909,Dangerous Minds
3,32,4.0,9909,Dangerous Minds
4,36,3.0,9909,Dangerous Minds
...,...,...,...,...
98550,664,3.0,34812,H.G. Wells' War of the Worlds
98551,664,4.5,22825,The Box
98552,665,4.0,110972,Pie in the Sky
98553,665,3.0,26602,Summer Catch


### 1. 위의 df를 사용하여 item-based cf에 필요한 평점으로 이루어진 데이터 df2 를 만드세요.  (결측값은 0으로) <font color=blue>(10점)</font>

In [104]:
df2= df.copy().pivot_table('rating', index= "id", columns= "userId")
df2.fillna(0, inplace=True)  #fillna: 누락된 값은 0으로 채우고 inplace=True로 df2에 변경사항을 저장하라는 것
df2

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
id,,,,,,,,,,,,,,,,,,,,,
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,5.0,0.0,0.0,5.0,3.5,0.0,0.0,...,0.0,0.0,4.0,3.0,0.0,0.0,3.0,0.0,0.0,4.0
12,0.0,0.0,3.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399106,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
401387,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
402672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. df2의 데이터를 사용하여 코사인 유사도 cos_matrix_2 를 구하여 데이터 프레임 df2_cosine을 만드세요.<font color=blue>(10점)</font><br>(인덱스, 컬럼 모두 df2의 인덱스로)

In [105]:
cos_matrix_2 = cosine_similarity(df2.values, df2.values)
cos_matrix_2

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.03860457,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.03860457, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [107]:
df2_cosine=pd.DataFrame(data=cos_matrix_2, index = df2.index, columns = df2.index)
df2_cosine

id,2,5,6,11,12,13,14,15,16,18,...,376570,384798,387893,390989,391698,399106,401387,402672,410921,416437
id,,,,,,,,,,,,,,,,,,,,,
2,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.226779,0.000000,...,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
5,0.0,1.000000,0.000000,0.197563,0.099554,0.150947,0.098432,0.079485,0.055027,0.169195,...,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.038605,0.0
6,0.0,0.000000,1.000000,0.087767,0.000000,0.122642,0.074557,0.100317,0.000000,0.087997,...,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0
11,0.0,0.197563,0.087767,1.000000,0.399134,0.547707,0.465270,0.349091,0.169882,0.477609,...,0.064620,0.05744,0.0,0.043080,0.0,0.0,0.071800,0.0,0.040616,0.0
12,0.0,0.099554,0.000000,0.399134,1.000000,0.442042,0.371946,0.199155,0.153964,0.345131,...,0.095659,0.00000,0.0,0.035872,0.0,0.0,0.023915,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399106,0.0,0.000000,0.000000,0.000000,0.000000,0.040461,0.048401,0.129413,0.000000,0.000000,...,0.000000,0.00000,0.0,0.000000,1.0,1.0,0.000000,0.0,0.000000,0.0
401387,0.0,0.000000,0.000000,0.071800,0.023915,0.013487,0.064535,0.129413,0.100791,0.024596,...,0.000000,0.00000,0.0,0.000000,0.0,0.0,1.000000,0.0,0.000000,0.0
402672,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.0,0.000000,0.0,0.0,0.000000,1.0,0.000000,0.0


## 3.item-based CF 평점 공식을 이용하여 예측 평점을 계산하세요
$$ \hat{r}_{5,i}  =  \frac{\sum_{j\in I_5}{sim(i,j)}\cdot r_{5,j}}{\sum_{j\in I_5}{sim(i,j)}} $$

* $\hat{r}_{5,i}$ 사용자 5가 영화 i에 남길 평점
* $r_{5,j}$ 사용자 5가 영화 j에 남긴 평점
* $sim(i,j)$ 영화 i와 j의 유사도
* $I_5$ 사용자 5가 평점을 남긴 영화 전체 집합 

### 3-(1) train_rate에서 유저 5의 데이터를 user5_train으로 만들어주세요.<font color=blue>(5점)</font>

In [108]:
user5_train= train_rate[train_rate['userId'] == 5][['id', 'rating']] 
user5_train

,id,rating
376,639,4.0
392,8916,4.0
373,510,3.0
423,1430,3.5
396,817,4.0
...,...,...
415,8835,4.5
404,1213,3.5
377,621,5.0
422,4147,4.0


### 3-(2) 공식의 분모인 유사도 합을 구하세요. <font color=blue>(20점)</font>

In [109]:
user5_train['id'].values

array([  639,  8916,   510,  1430,   817,   812,   426,    69,   568,
        1640,   118,  8193,  6957,  9522, 11003,   863,  9377,   788,
        8467, 15121,   423,   330,  9032,   392,   170,   462,  2105,
        3049,  9327,   557,   238,  9489,  9603, 11000,   630,  2770,
        8346,   205,  1624,   786,   254,   455,   114,  9614,   597,
         565, 15602,   745,    70,  3981,  1597,  9390,  9372,  2109,
         854, 11224, 11566,  8587,   544,   453,   153,   411,  2898,
        2048,   808,   380,   492,  2108,  1637,  1542, 10020,   242,
         787,  2024,  8835,  1213,   621,  4147,    12], dtype=int64)

In [110]:
#user5_train의 79개 영화인 j와 전체 영화 i 에 대해)
sim_sum = df2_cosine.loc[user5_train['id'].values, :].sum().values      #유사도 합
sim_sum = sim_sum + 1                                                 #분모가 0인경우 발생할 계산오류를 피하기 위해 +1 해줌
sim_sum #numpy 배열로 저장된 상태

array([ 1.38829044, 10.91533732,  6.65166546, ...,  1.30397893,
        4.53056159,  2.42612583])

### 3-(3) 공식의 분자를 계산해서 sim_rating에 할당해주세요.<font color=blue>(20점)</font>

In [112]:
df2_cosine.loc[user5_train['id'].values, :].T.values.shape

(8834, 79)

In [113]:
user5_train['rating'].values.shape

(79,)

In [114]:
#공식의 분자 계산 - 각각 곱하고 더한 값이 필요하므로 내적 함수사용  
sim_rating = np.matmul(df2_cosine.loc[user5_train['id'].values, :].T.values, user5_train['rating'].values)
sim_rating

array([ 1.28562007, 39.05991653, 22.01407663, ...,  1.26279642,
       13.65901885,  5.43237159])

### 3-(4) 구한 분모, 분자 값을 이용하여 평점 예측 값 pred_rating 데이터 프레임을 만들어 주세요.<font color=blue>(15점)</font> <br>(인덱스는 df2_cosine의 인덱스)

In [115]:
pred_rating = pd.DataFrame(np.divide(sim_rating, sim_sum), index=df2_cosine.index)
pred_rating.columns=["pred"]
pred_rating 

,pred
id,
2,0.926045
5,3.578443
6,3.309559
11,3.763187
12,3.811819
...,...
399106,3.393501
401387,3.304334
402672,0.968418


### 4. test_rate에서 유저5의 데이터를 user5_test에 할당해주세요.<font color=blue>(5점)</font>

In [116]:
#이전에는 train_rate, 즉 train set를 기반으로 평점 예측을 한 것이고 이번에 다루는 것은 test set의 데이터이다
user5_test= test_rate[test_rate['userId'] == 5][['id', 'rating']]
user5_test

,id,rating
436,558,4.5
428,310,4.0
374,240,2.5
417,161,3.0
425,672,3.0
375,37247,4.0
411,8358,3.5
414,824,3.5
412,1493,4.5
388,9552,3.5


### 5. user5_test와 pred_rating 데이터프레임을 병합해 아래의 결과처럼 나오도록 해주세요.<font color=blue>(5점)</font>

In [118]:
item_cf = pd.merge(user5_test, pred_rating.loc[user5_test['id']], left_on= 'id', right_index=True)
item_cf.head()

,id,rating,pred
436,558,4.5,3.807301
428,310,4.0,3.784333
374,240,2.5,3.700531
417,161,3.0,3.809754
425,672,3.0,3.794419


### 6. 앞에서 만들었던 평가지표 함수를 사용하여 rmse, mae 결과를 출력하세요.<font color=blue>(5점)</font>

In [119]:
evaluation(item_cf['rating'], item_cf['pred'])

RMSE: 0.588
MAE: 0.486


### 7. similar movie 함수를 이용하여 Avatar와 2번째로 유사한 영화 제목과 예상 평점을 마크다운으로 작성해주세요.<font color=blue>(5점)</font><br>(유사도는 cos_matirx_2 사용)

In [120]:
similar_movie("Avatar",cos_matrix_2)

                                           유사한 영화 TOP10      예상평점
8128                                The Kings of Summer  3.284065
8587                                           The Loft  3.321532
7497  The Chronicles of Narnia: The Voyage of the Da...  2.842588
7072                                    The Hurt Locker  3.645068
7752                                     Violet & Daisy  3.321532
4925                                        Naked Lunch  3.376687
1714                          The Man Who Knew Too Much  3.597158
4256                                    Far from Heaven  3.589522
8244                    The Hunger Games: Catching Fire  3.435007


유사한 영화: The Loft <br>
예상 평점: 3.321532